In [1]:
import torch
import torch.nn as nn
import numpy as np
torch.__version__

'1.6.0'

# 3.1 logistic回归实战

处理结构化数据，用logistic回归对结构化数据简单分类

## 3.1.1 logistic回归介绍

logistic回归是一种广义线性回归（generalized linear model），与多重线性回归分析有很多相同之处。
模型形式基本相同，都具有wx + b，w和b是待求参数。区别在于因变量不同，多重线性回归直接将wx+b作为因变量，即y=wx+b。
而logistic回归通过函数*L*将  wx+b对应p=L(wx+b), 根据p与1-p的大小决定因变量的值。
如果L是logistic函数就是logistic回归，如果L是多项式函数就是多项式回归。

logistic回归在线性回归后再调用一层logistic函数。
logistic回归主要进行二分类预测，Sigmod函数是最常见的logistic函数。
因为Sigmod函数输出0~1间的概率值，概率大于0.5预测为1，小于0.5预测为0。

下面使用公开数据进行介绍

## 3.1.2 UCI German Credit  数据集

UCI German Credit是UCI德国信用数据集，里面有原数据和数值化后的数据。

German Credit数据集根据个人银行**贷款信息**和申请客户**贷款逾期**情况预测**贷款违约倾向**，
包含24维的1000条数据，用处理好的数值化数据展示。
[地址](https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/)

## 3.2 代码实战
german.data-numeric是numpy处理好的数值化数据，直接用numpy的load方法读取

In [2]:
data = np.loadtxt("german.data-numeric")

数据读取完成后对数据做归一化处理

In [3]:
n, l = data.shape
print(n, l)
data[:5][-1]

1000 25


array([ 1., 24.,  3., 49.,  1.,  3.,  3.,  4.,  4., 53.,  3.,  2.,  2.,
        1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  2.])

In [4]:
for j in range(l - 1):
    meanVal = np.mean(data[:, j])
    stdVal = np.std(data[:, j])
    data[:, j] = (data[:, j] - meanVal) / stdVal

打乱数据

In [5]:
np.random.shuffle(data)

区分训练集和测试集，没有验证集，使用测试集准确度作为评判好坏标准

800条用于训练，200条作为测试

german.data-numeric格式: 前24列为24个维度，最后一个为标签（0，1）

In [16]:
train_data = data[:800, :l - 1]
train_label = data[:800, l - 1] - 1

test_data = data[800:, :l - 1]
test_label = data[800:, l - 1] - 1

train_label[:5].dtype
test_label[:5].dtype

dtype('float64')

定义模型

In [36]:
class LR(nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        # 由于24个维度已经固定，这里写24
        self.fc = nn.Linear(24, 2)  
        

    def forward(self, x):
        out = self.fc(x)
        out = torch.sigmoid(out)
        return out

测试集准确率

In [8]:
def test(pred, label):
    t = pred.max(-1)[1] == label
    return torch.mean(t.float())

一些设置

In [30]:
net = LR()

# CrossEntropyLoss损失
criterion = nn.CrossEntropyLoss()

# Adam优化
optm = torch.optim.Adam(net.parameters())
# optm = torch.optim.RMSprop(net.parameters())

开始训练

In [14]:
test_label

tensor([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 0])

In [37]:
# 输入值转化成torch的Tensor
X_train = torch.from_numpy(train_data).float()
y_train = torch.from_numpy(train_label).long()

X_test = torch.from_numpy(test_data).float()
y_test = torch.from_numpy(test_label).long()

# 训练5000次
epochs = 5000

for i in range(epochs):

    # 指定模型为训练模式，计算梯度
    net.train()
    y_hat = net(X_train)

    # 计算损失
    loss = criterion(y_hat, y_train)

    # 前一步的损失清零
    optm.zero_grad()

    # 反向传播
    loss.backward()

    # 优化
    optm.step()

    # 每100次输出相关信息
    if (i + 1) % 500 == 0:

        # 指定模型为计算模式
        net.eval()
        y_pred = net(X_test)

        # 计算准确率
        accu = test(y_pred, y_test)
        print("Epoch:%d,Loss:%4f,Accuracy：%2f" % (i + 1, loss.item(), accu))

Epoch:500,Loss:0.503469,Accuracy：0.775000
Epoch:1000,Loss:0.502480,Accuracy：0.770000
Epoch:1500,Loss:0.501607,Accuracy：0.770000
Epoch:2000,Loss:0.500800,Accuracy：0.770000
Epoch:2500,Loss:0.500053,Accuracy：0.775000
Epoch:3000,Loss:0.499360,Accuracy：0.775000
Epoch:3500,Loss:0.498713,Accuracy：0.775000
Epoch:4000,Loss:0.498110,Accuracy：0.775000
Epoch:4500,Loss:0.497544,Accuracy：0.775000
Epoch:5000,Loss:0.497014,Accuracy：0.775000


训练完成,准确度达到78%